In [1]:
import collections
import datetime
import dateutil
import html
import pathlib
from dotenv import dotenv_values
import pandas as pd

from grouptag import whichtag, quicktag, VectorTagger, PatternTagger, fix_ruleframe


from territories import fix_raw_countries, territories_qtspec
from acronyms import acromap

In [2]:
pd.set_option('max_colwidth', None)

In [3]:
config = dotenv_values('config.env')

# Read cached

In [4]:
%%time

raw_df = pd.read_feather(config.get('PATH_DF_FULL'))
raw_df.shape

CPU times: total: 21.4 s
Wall time: 1min 16s


(835499, 63)

# Cleaning / Conversion

## title

Mixture of `public_title` and `scientific_title`

In [5]:
na_values = {'a - a', 'na - na', 'asdf', 'none', 'n.a.',
             'not applicable', 'not available', 'see below'}

title_spec = [  
    [pd.NA, lambda x: len(x) <= 2],
    {'series.lower': True},
    [pd.NA, na_values]
]

def clean_title(series):
    unescaped = series.apply(html.unescape)
    # No non-word symbols in the beginning of the line
    cleaned = unescaped.str.replace(r'^\W+', '', regex=True)
    # One non-word symbol (puntuation mark?) allowed after the word in the end
    cleaned = cleaned.str.replace(r'(\w\W)\W+$', r'\1', regex=True)
    result = cleaned.str.replace(r'N/A[\s\-]*', '', case=False, regex=True)
    result = whichtag(result, title_spec, prefill=result)
    return result

In [6]:
%%time

scientific = clean_title(raw_df['scientific_title'])
public = clean_title(raw_df['public_title'])
title = public.fillna(scientific)

CPU times: total: 21.6 s
Wall time: 28.2 s


In [7]:
title.isna().value_counts()

False    835295
True        204
Name: public_title, dtype: int64

## url

In [8]:
raw_df['url'].isna().value_counts()  # leave as is

False    835499
Name: url, dtype: int64

### study_type

In [9]:
%%time

na_values = {'', 'N/A', 'Unknown', 'Not Specified'}

study_type_spec = [  
    [pd.NA, na_values],
    {'series.lower': True},
    ['Interventional/Observational', 'interven', 'observ'],
    ['Diagnostic Test', 'diagnostic'],
    ['Cause/Relative Factors', ['cause', ['factor', 'relative']]],
    ['Interventional', ['interven', 'treatment']],
    ['Observational', 'observ'],
    ['Post-Marketing', 'pms'],
    ['Basic Science/Epidemiological', ['basic', 'epidemi']],
    ['Prognistic', 'progn'],
    ['Health Services Research', 'health', 'services', 'reaserch'],
    [pd.NA, 'other'],
]

raw_study_type = raw_df['study_type']
study_type = whichtag(raw_study_type, study_type_spec, prefill=raw_study_type)
study_type.value_counts(dropna=False)

CPU times: total: 3.14 s
Wall time: 4.06 s


Interventional                   666281
Observational                    156748
Diagnostic Test                    3053
Cause/Relative Factors             2307
<NA>                               1916
Basic Science/Epidemiological      1791
Expanded Access                     870
Post-Marketing                      657
BA/BE                               594
Interventional/Observational        487
Prognistic                          247
Health Services Research            213
Prevention                          183
Screening                           152
Name: study_type, dtype: Int64

## phase

In [10]:
%%time

na_values = {'', 'n/a', 'nill', 'null', 'na', 'other', 'not entered',
             'not specified', 'not selected', 'not applicable', 'not available'}
clean_spec = [{'series.lower': True}, [pd.NA, na_values]]
raw_phase = whichtag(raw_df['phase'], clean_spec, prefill=raw_df['phase'])

CPU times: total: 406 ms
Wall time: 622 ms


In [11]:
%%time

phase4_spec = [
    {'patterns.lower': True, 'series.lower': True,
     1: {'str.contains': {'case': True, 'regex': True}}},
    [True, {'Human pharmacology': 'str.startswith'}, '(Phase IV): yes'],
    [False, {'Human pharmacology': 'str.startswith'}],
    [True, {r'\b4\b': 1}],
    [True, 'Phase4'],
    [True, 'post', 'market'],
    {'patterns.lower': False, 'series.lower': False},
    [True, {r'\bIV\b': 1}],
]

phase3_spec = [
    {'patterns.lower': True, 'series.lower': True,
     1: {'str.contains': {'case': True, 'regex': True}}},
    [True, {'Human pharmacology': 'str.startswith'}, '(Phase III): yes'],
    [False, {'Human pharmacology': 'str.startswith'}],
    [True, {r'\b3[ab]?\b': 1}],
    [True, 'Phase3'],
    {'patterns.lower': False, 'series.lower': False},
    [True, {r'\bIII[abAB]?\b': 1}],
]

phase2_spec = [
    {'patterns.lower': True, 'series.lower': True,
     1: {'str.contains': {'case': True, 'regex': True}}},
    [True, {'Human pharmacology': 'str.startswith'}, '(Phase II): yes'],
    [False, {'Human pharmacology': 'str.startswith'}],
    [True, {r'\b2[ab]?\b': 1}],
    [True, 'Phase2'],
    {'patterns.lower': False, 'series.lower': False},
    [True, {r'\bII[abAB]?\b': 1}],
]

phase1_spec = [
    {'patterns.lower': True, 'series.lower': True,
     1: {'str.contains': {'case': True, 'regex': True}}},
    [True, {'Human pharmacology': 'str.startswith'}, '(Phase I): yes'],
    [False, {'Human pharmacology': 'str.startswith'}],
    [True, {r'\b1[ab]?\b': 1}],
    [True, 'Phase1'],
    {'patterns.lower': False, 'series.lower': False},
    [True, {r'\bI[abAB]?\b': 1}],
]

phase0_spec = [[True, ['0', 'zero']]]

phase4 = whichtag(raw_phase, phase4_spec).fillna(False)
phase3 = whichtag(raw_phase, phase3_spec).fillna(False)
phase2 = whichtag(raw_phase, phase2_spec).fillna(False)
phase1 = whichtag(raw_phase, phase1_spec).fillna(False)
phase0 = whichtag(raw_phase, phase0_spec).fillna(False)

CPU times: total: 7.23 s
Wall time: 9.16 s


In [12]:
def append_value(x, y):    
    cell = list() if x is pd.NA else x
    cell.append(int(y))
    return cell

In [13]:
%%time

phases = pd.Series(pd.NA, index=raw_df.index)
phases[phase0] = phases[phase0].apply(append_value, y=0)
phases[phase1] = phases[phase1].apply(append_value, y=1)
phases[phase2] = phases[phase2].apply(append_value, y=2)
phases[phase3] = phases[phase3].apply(append_value, y=3)
phases[phase4] = phases[phase4].apply(append_value, y=4)
phases.value_counts(dropna=False)

CPU times: total: 1.08 s
Wall time: 1.6 s


<NA>         485287
[2]           93688
[3]           92865
[4]           55247
[1]           53746
[1, 2]        20557
[0]           16529
[2, 3]        15779
[3, 4]         1582
[1, 3]          127
[1, 2, 3]        51
[1, 4]           23
[2, 4]           12
[2, 3, 4]         3
[1, 2, 4]         2
[1, 3, 4]         1
dtype: int64

## enrollment_date

In [14]:
%%time

enrollment_date_str = raw_df['enrollment_date_str']

# Fix bad formats, assign 15 as missing day of month
enrollment_date_str = (
    enrollment_date_str
    .str.replace(r'^(\d{4})(\d{2})(\d{2})$', r'\1-\2-\3', regex=True)  # 20221001
    .str.replace(r'^(\w+)\s+(\d{4})$', r'\1 15, \2', regex=True)  # like October 2022
    .str.replace(r'^(\d{1,2})/(\d{4})$', r'15/\1/\2', regex=True)  # like 10/2022
)
enrollment_date = (
    pd.to_datetime(enrollment_date_str, dayfirst=True, errors='coerce')
    .dt.date
    .clip(datetime.date(1970,1,1), datetime.date(2099,12,31))
)

CPU times: total: 44.6 s
Wall time: 59.6 s


## registration_date

In [15]:
%%time

registration_date = (
    pd.to_datetime(raw_df['registration_date_str'], 
                   dayfirst=True).dt.date.copy()
)
registration_date_valid = ~(
    registration_date.isna() |
    (registration_date == datetime.date(1990,1,1))
)
registration_date = registration_date.where(registration_date_valid, enrollment_date)

CPU times: total: 406 ms
Wall time: 929 ms


## target_size

In [16]:
def to_numeric(series):
    return (
        pd.to_numeric(series, errors='coerce')
        .convert_dtypes(convert_integer=True)
        .astype('Int32')
    )

In [17]:
%%time

target_size_str = raw_df['target_size']
target_size = pd.Series(None, index=target_size_str.index, dtype='Int32')

# Pattern for "each group has its size":
# <group 1 description>:###;<group 2 description>:###;...;
target_size_group_df = (
    target_size_str
    .str.extractall(r'.*:(\d+);')
    .reset_index(level=-1, drop=True)
    .assign(size=lambda x: to_numeric(x[0]))
    .groupby(level=0)
    .agg({'size': 'sum'})
)
target_size[target_size_group_df.index] = target_size_group_df['size']
target_size_str = target_size_str[target_size.isna()]

# Simple extraction
extracted = (
    target_size_str
    .str.lower()
    .replace(r'^9{7,}$', '', regex=True)
    .replace(r'^(\d+)[\s\-\(,].*$', r'\1', regex=True)
    .replace(r'^(total|minimum|at least):?\s+(\d+)$', r'\2', regex=True)
    .replace(r'^(in total|total|global|until)[^\d]+(\d+).*$', r'\2', regex=True)
    .replace(r'^[\w\s]+:\s*(\d+);$', r'\1', regex=True)
)
target_size_extracted = to_numeric(extracted).clip(0)
target_size[target_size_extracted.index] = target_size_extracted

CPU times: total: 8.95 s
Wall time: 12.9 s


## recruitment_status

In [18]:
%%time

raw_recruitment_status = raw_df['recruitment_status']

recruitment_status_qtspec = {    
    'Not Recruiting': ['pending', ['recruit', ['not', 'close', 'complet', 'no longer']],
                      ['without', 'enroll']],
    'Completed': ['complet', 'publish'],
    'In Expanded Access': ['available', 'marketing'],
    'Closed/Suspended': ['withdrawn', 'withheld', 'terminated', 'suspend', 'closed', 'stop', 'halt'],
    'Open': ['authori', 'ongo'],
    'Recruiting': ['recruit', 'invitation']
}

recruitment_status = quicktag(raw_recruitment_status,
                              rules=recruitment_status_qtspec,
                              case=False, regex=False)

vc = recruitment_status.value_counts(dropna=False)
vc

CPU times: total: 3.08 s
Wall time: 4.53 s


Completed             356566
Not Recruiting        209916
Recruiting            178585
Closed/Suspended       43244
Open                   33673
<NA>                   12645
In Expanded Access       870
dtype: int64

## primary_sponsor

In [19]:
ruleframes_path = config.get('PATH_RULEFRAMES')
sheet_name = config.get('SHEET_NAME_PRIMARY_SPONSOR')

ruleframe = pd.read_excel(ruleframes_path, sheet_name=sheet_name, header=0)

In [20]:
fix_ruleframe(ruleframe)

,tag,phrase,min_terms
0,Novartis,Novartis Sandoz Hexal Endocyte Corthera,1
1,GSK,GlaxoSmithKline GSK,1
2,GSK,Stiefel +Laboratories,2
3,GSK,Human Genome Sciences Inc,6
4,GSK,TESARO ViiV,1
...,...,...,...
100,Camurus,Camurus,1
101,Biohaven,Biohaven,1
102,Crinetics,Crinetics,1
103,Aeterna Zentaris,Aeterna Zentaris,3


In [21]:
%%time

primary_sponsor = raw_df['primary_sponsor']

stop_words = ['an', 'and', 'as', 'by', 'for', 'from',
              'of', 'or', 'the', 'to', 'with']
tagger = VectorTagger(stop_words=stop_words)
tagger = tagger.fit(primary_sponsor.apply(html.unescape))

CPU times: total: 12.3 s
Wall time: 17 s


In [22]:
%%time

sponsor_tag = tagger.transform(ruleframe)
vc = sponsor_tag.value_counts()
vc[:10]

CPU times: total: 1 s
Wall time: 1.35 s


Novartis                11140
GSK                      7805
Pfizer                   7799
AstraZeneca              7227
Roche                    6708
Merck (MSD)              6278
Johnson & Johnson        5986
Sanofi                   5201
Bristol Myers Squibb     5131
Boehringer Ingelheim     4416
dtype: int64

## countries

In [23]:
def set2list(x):
    return list(x) if isinstance(x, set) else x

In [24]:
%%time

raw_countries = fix_raw_countries(raw_df['countries'])
countries = pd.Series(None, dtype='object', index=raw_df.index)
exploded = raw_countries.str.findall(r'[^,;]+').explode().str.strip()
unique_territories = pd.Series(exploded.unique()).fillna('')
territories_tag = quicktag(unique_territories, territories_qtspec,
                           prefill=pd.NA, case=False, regex=True)
exploded_map = (
    pd.DataFrame({
        'territories_tag': territories_tag,
        'territories': unique_territories
    })
    .dropna()
    .set_index('territories')
    ['territories_tag'].to_dict()
)
county_sets = (
    exploded
    .map(exploded_map)
    .dropna()
    .groupby(level=0)
    .agg(set)
)
countries[county_sets.index] = county_sets.values
countries = countries.apply(set2list)

CPU times: total: 51.1 s
Wall time: 1min 4s


## conditions (medarea)

In [25]:
def is_informative(series):
    return ~(
        series.isna() |
        series.str.lower().isin(['', 'null', 'none', 'n/a',
                                 'not applicable',
                                 'pharmacokinetics', 'bioequivalence', 
                                 'investigations', 'vaccination', 'immunization']) |
        series.str.startswith('<Manually entered code') |
        series.str.startswith('-') |
        series.str.match(r'^\w\d+$', case=False) |
        series.str.match(r'^healthy(\s\w+){0,2}$', case=False) |
        series.str.contains('hygiene', case=False)
    )

In [26]:
def replace_known_acronyms(series, acromap):
    # Acronym is an all-caps single word
    # All unknown acronyms are dropped
    is_acronym = (series == series.str.upper()) & ~series.str.contains(r'\s', regex=True)
    return (
        series
        .map(acromap)
        .fillna(series.where(~is_acronym))
        .reset_index()
        .drop_duplicates()
        .set_index('index')
        .groupby(level=0)
        .agg(list)
        [series.name]
        .str.join('; ')
    )

### Pre-clean conditions

In [27]:
%%time
raw_conditions = raw_df['conditions']

# Clean html escapes and <br> tags
conditions = (
    raw_conditions
    .apply(html.unescape)
    .str.replace('<br>', '', case=True, regex=False)
    .str.strip()
    .str.strip(',;')
)

conditions.size, conditions.index.nunique()

CPU times: total: 1.47 s
Wall time: 1.82 s


(835499, 835499)

### Extract MedDRA Terms

In [28]:
%%time
# MedDRA records
is_meddra_record = conditions.str.contains('meddra', case=False, regex=False)
meddra = conditions[is_meddra_record]
for name in ['MedDRA', 'Level:', 'Term:', 'Classification code', 
             'System Organ Class:', 'Therapeutic area:', 'Preferred term:']:
    meddra = meddra.str.replace(name, f'; {name}', case=True, regex=False)
meddra = meddra + ';'
meddra_term_text = pd.Series(pd.NA, index=meddra.index, name='term')  # placeholder
# extract terms, replace known acronyms and build back
extracted_term = (
    meddra.str.extractall(r'Term:\s*([^;]+);')
    .reset_index(level=1, drop=True)
    [0]
    .str.strip(r' \t;')
)
meddra_term = replace_known_acronyms(extracted_term, acromap)
# Fill in improper values with MedDRA description
meddra_term_text[meddra_term.index] = meddra_term.values
meddra_desc = meddra.str.split(';').str[0].str.strip()
meddra_term_text = meddra_term_text.where(is_informative(meddra_term_text), meddra_desc)

CPU times: total: 5.16 s
Wall time: 6.13 s


### Extract Non-MedDRA Terms

In [29]:
%%time
# Non-MedDRA records
# Get rid of Therapeutic area info
non_meddra = (
    conditions[~is_meddra_record]
    .str.extract(r'(.*);Therapeutic area:', expand=False)
    .str.strip(r' \t;')
    .fillna(conditions[~is_meddra_record])
)
non_meddra_term_text = pd.Series(pd.NA, index=conditions[~is_meddra_record].index, name='term')  # placeholder
# extract terms, replace known acronyms, throw out non-informatives and build back
non_meddra_exploded = (
    non_meddra
    .str.split(';')
    .explode()
)
extracted_term = (
    non_meddra_exploded.where(is_informative(non_meddra_exploded))
    .dropna()
    .str.strip(r' \t;')
)
non_meddra_term = replace_known_acronyms(extracted_term, acromap)
# Use title for non-informative records
non_meddra_term_text[non_meddra_term.index] = non_meddra_term.values

CPU times: total: 1min 31s
Wall time: 1min 47s


### Put all terms together

In [30]:
term_text = pd.Series(pd.NA, index=conditions.index, name='term')
for term_series in (meddra_term_text, non_meddra_term_text):
    term_text[term_series.index] = term_series.values
term_text.fillna('', inplace=True)
assert term_text.index.equals(conditions.index)

### Tag using Vectortag

In [31]:
%%time
stop_words = ['and', 'after', 'for', 'from', 'the', 'with']
tagger = VectorTagger(stop_words=stop_words,
                      strip_accents='unicode',
                      token_pattern=r'\b[^\W\d][\w-]{2,}\b', 
                      ngram_range=(1, 2))
tagger = tagger.fit(term_text)

ruleframes_path = config.get('PATH_RULEFRAMES')
sheet_name = config.get('SHEET_NAME_CONDITIONS')

ruleframe = pd.read_excel(ruleframes_path, sheet_name=sheet_name, header=0)
medarea_tag = tagger.transform(ruleframe)

CPU times: total: 17.8 s
Wall time: 21.4 s


### Tag Title using Vectortag

Use title as a source of terms where medarea tag is not assigned.

In [32]:
%%time
title_text = title[medarea_tag.isna()].fillna('')
tagger = tagger.fit(title_text)
medarea_title_tag = tagger.transform(ruleframe)
medarea_tag[medarea_title_tag.index] = medarea_title_tag.values

CPU times: total: 9.55 s
Wall time: 11.6 s


# Conditions Fixing Block

In [33]:
focus_sponsors = ['Teva', 'Corbus', 'Enlivex', 
                  'Ascendis', 'Camurus', 'Biohaven', 'Crinetics', 'Aeterna Zentaris', 'Merck (KGaA)', 'Merck (MSD)'
                 ]
is_focus = ( 
    sponsor_tag.isin(focus_sponsors) |
    raw_df['public_contact_email'].str.contains('Accelsiors', case=False, regex=False) |
    raw_df['scientific_contact_email'].str.contains('Accelsiors', case=False, regex=False)
) & medarea_tag.isna()

In [34]:
is_focus.value_counts()

False    834975
True        524
dtype: Int64

In [35]:
term_text[is_focus].value_counts().size

143

# Get resulting dataset

In [36]:
prefiltred_df = pd.DataFrame({
    'trial_id': raw_df['trial_id'],
    'url': raw_df['url'],
    'title': title,    
    'registration_date': registration_date,
    'study_type': study_type,
    'target_size': target_size,
    'phases': phases,
#     'phase0': phase0,
#     'phase1': phase1,
#     'phase2': phase2,
#     'phase3': phase3,
#     'phase4': phase4,
    'recruitment_status': recruitment_status,
    'medarea': medarea_tag,
    'sponsor': sponsor_tag,    
    'primary_sponsor': raw_df['primary_sponsor'],
    'secondary_sponsors': raw_df['secondary_sponsors'],    
    'countries': countries    
})

In [37]:
%%time

# Filter off "children"
child_bridged_type = raw_df[raw_df['bridged_type'] == 'child']['trial_id'].values
children_ids = (
    raw_df[raw_df['children'] != 'NULL']['children']
    .str.split(';')
    .explode()
    .values
)
children = set(child_bridged_type) | set(children_ids)
cleaned_df = prefiltred_df[~prefiltred_df['trial_id'].isin(children)]

CPU times: total: 14.4 s
Wall time: 41.6 s


In [38]:
cleaned_df.shape

(755980, 13)

# Keep only known sponsor  records

In [39]:
selected_df = cleaned_df[~cleaned_df['sponsor'].isna()]
selected_df.shape

(65854, 13)

# Save selected

In [40]:
%%time
selected_df_path = config.get('PATH_DF_SELECTED')
selected_df.reset_index(drop=True).to_feather(selected_df_path)

CPU times: total: 406 ms
Wall time: 1.41 s


In [41]:
selected_df['medarea'].value_counts(dropna=False)

Oncology                 13651
Endocrine/Nutritional     7048
<NA>                      6956
Infectious/Parasitic      4910
Cardiovascular            4439
Respiratory               4262
Nervous                   3132
Musculoskeletal           3030
Mental/Behavioural        2800
Digestive                 2660
Skin                      2637
Genitourinary             2485
Visual                    1743
Pain                      1481
Blood                     1393
HIV/AIDS                  1110
Immune                     762
Sexual                     389
Sleep/Wake                 340
COVID-19                   255
Injury                     202
Reproductive               169
Name: medarea, dtype: int64